This notebook collects all the game ID's from 

In [8]:
import json
import pandas as pd

all_games = "http://api.steampowered.com/ISteamApps/GetAppList/v0002/?key=STEAMKEY&format=json"


In [21]:
with urllib.request.urlopen(all_games) as url:
    df = pd.json_normalize(json.load(url)["applist"], record_path =['apps'])



df

,appid,name
0,1941401,
1,2241530,Adventure Awaits Demo
2,2241780,WildArea Demo
3,2241190,Christmas Luge
4,2239050,Poly Puzzle: Animals
...,...,...
151510,1316840,theHunter: Call of the Wild™ - Silver Ridge Peaks
151511,793360,Big Crown Showdown
151512,2151420,Word
151513,1364390,Animaze
